# SQL Server Auth – Komplettbeispiele (7 Methoden)

Dieses Notebook zeigt **alle unterstützten Authentifizierungsarten** des Microsoft ODBC Drivers mit unserer `build_engine(...)`/`load_sql_settings(...)`-API.

**Voraussetzungen**
- Treiber: *ODBC Driver 17 oder 18 for SQL Server* auf dem System installiert
- Python-Pakete: `sqlalchemy`, `pyodbc`, `pandas`
- Repo-Pfad in `sys.path` (eine Ebene hoch)
- Eine `config.json` mit passenden Knoten (je Methode ein eigener Knoten)

Die 7 Methoden (jeweils **eigener Knoten**):
1. `connections.sql_basic` – SQL Server Authentication (User/Pass)
2. `connections.sql_trusted` – Windows Integrated / Trusted Connection
3. `connections.sql_aad_password` – Azure AD Password (`ActiveDirectoryPassword`)
4. `connections.sql_aad_integrated` – Azure AD Integrated (`ActiveDirectoryIntegrated`)
5. `connections.sql_aad_interactive` – Azure AD Interactive (`ActiveDirectoryInteractive`)
6. `connections.sql_aad_msi` – Azure AD Managed Identity (`ActiveDirectoryMsi`)
7. `connections.sql_aad_sp` – Azure AD Service Principal (`ActiveDirectoryServicePrincipal`)


In [ ]:
from pathlib import Path
import sys, importlib
sys.path.insert(0, "..")  # Repo-Root

import graphfw as graphfw, graphfw.core, graphfw.core.config as cfg
importlib.invalidate_caches()
_ = importlib.reload(graphfw)
_ = importlib.reload(graphfw.core)
cfg = importlib.reload(cfg)

from graphfw.core.config import load_sql_settings  # <- sollte jetzt klappen


## Hilfsfunktionen

In [7]:
def quick_check(engine):
    """Führt SELECT 1 aus und gibt das Ergebnis zurück."""
    with engine.connect() as c:
        return c.execute(text("SELECT 1")).scalar()


def show_settings(settings):
    d = settings.as_dict(mask_secrets=True)
    print({k: d[k] for k in ("server","db_name","username","driver","params") if k in d})

def connect_and_check(node: str, *, show_drivers: bool = True, show_dsns: bool = True):
    print(f"\n=== Node: {node} ===")
    if show_drivers:
        print("ODBC-Treiber (SQL Server):", list_odbc_drivers())
    if show_dsns:
        print("ODBC-DSNs:", list_odbc_data_sources())

    settings, info = load_sql_settings(config_path=CONFIG_PATH, node=node, env_override=True)
    print("Quelle:", info.get("source"), "| Node:", info.get("node_path"))
    print("Settings:", settings.as_dict(mask_secrets=True))

    ok, diag = diagnose_with_fallbacks(settings)
    print(diag["summary"])
    for i, att in enumerate(diag["attempts"], 1):
        status = "OK" if att.get("ok", False) else ("OK" if att.get("method","").startswith(("pyodbc","ado","pymssql")) and "error" not in att else "ERR")
        # kompaktes Protokoll
        print(f"  [{i:02d}] {att.get('method','?'):<18} | driver={att.get('driver') or att.get('provider')} "
              f"| params={att.get('params') or ''} | {att.get('duration_s','-')}s")
        if "error" in att and att["error"]:
            print("       ", att["error"])
    if diag.get("suggestions"):
        print("\nHinweise:")
        for s in diag["suggestions"]:
            print(" -", s)
    return ok, diag


## (Optional) Beispiel-Knoten anlegen

Falls deine `config.json` die Knoten noch **nicht** enthält, kannst du mit
`save_sql_settings(...)` Platzhalter anlegen. **Achtung:**
`WRITE_SKELETON=True` überschreibt vorhandene Knoten **nicht**, sondern ergänzt
nur fehlende Keys (siehe `overwrite=False`). Passe Werte manuell an.

In [11]:
CONFIG_PATH = r"C:\python\Scripts\config_sql.json"
WRITE_SKELETON = True  # auf True setzen, um Platzhalter anzulegen

if WRITE_SKELETON:
    examples = {
        "connections.sql_basic": {
            "server": "myserver.domain.tld",
            "db_name": "BI_RAW",
            "username": "svc_user",
            "password": "CHANGE_ME",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "sql",
            "params": {"TrustServerCertificate": "yes"}
        },
        "connections.sql_trusted": {
            "server": "myserver\\sql2019",
            "db_name": "BI_RAW",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "trusted",
            "params": {"TrustServerCertificate": "yes"}
        },
        "connections.sql_aad_password": {
            "server": "myserver.database.windows.net",
            "db_name": "BI_RAW",
            "username": "user@contoso.com",
            "password": "CHANGE_ME",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "aad-password",
            "params": {"Encrypt": "yes"}
        },
        "connections.sql_aad_integrated": {
            "server": "myserver.database.windows.net",
            "db_name": "BI_RAW",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "aad-integrated"
        },
        "connections.sql_aad_interactive": {
            "server": "myserver.database.windows.net",
            "db_name": "BI_RAW",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "aad-interactive"
        },
        "connections.sql_aad_msi": {
            "server": "myserver.database.windows.net",
            "db_name": "BI_RAW",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "aad-msi"
            # Optional (user-assigned MI): "params": {"ClientId": "<GUID>"}
        },
        "connections.sql_aad_sp": {
            "server": "myserver.database.windows.net",
            "db_name": "BI_RAW",
            "username": "00000000-0000-0000-0000-000000000000",  # App (Client) ID
            "password": "CHANGE_ME_SECRET",
            "driver": "ODBC Driver 18 for SQL Server",
            "auth": "aad-sp"
        }
    }

    for node, payload in examples.items():
        ok, sinfo = save_sql_settings(
            config_path=CONFIG_PATH,
            node=node,
            settings=payload,
            overwrite=False  # fehlende Keys ergänzen, bestehende nicht überschreiben
        )
        print(node, "->", ok, sinfo.get("error", "ok"))
else:
    print("Skelett-Anlage übersprungen (WRITE_SKELETON=False)")

connections.sql_basic -> True ok
connections.sql_trusted -> True ok
connections.sql_aad_password -> True ok
connections.sql_aad_integrated -> True ok
connections.sql_aad_interactive -> True ok
connections.sql_aad_msi -> True ok
connections.sql_aad_sp -> True ok


## 1) SQL Server Authentication (Benutzer/Pass) — `connections.sql_basic`

In [8]:
connect_and_check("connections.sql_basic")


=== Node: connections.sql_basic ===


NameError: name 'list_odbc_drivers' is not defined

In [30]:
from pathlib import Path
import sys
sys.path.insert(0, "..")  # Repo-Root (Ordner, der 'graphfw/' enthält)

from sqlalchemy import text
from graphfw.core.config import load_sql_settings
from graphfw.io.writers.sql_writer import build_engine  # dein sql_writer.py

CONFIG_PATH = Path("config.json")
settings, info = load_sql_settings(config_path=CONFIG_PATH, node="connections.sql_basic")
cfg, db, user, pwd = settings.to_engine_args()

engine = build_engine(cfg, db_name=db, username=user, password=pwd)
with engine.connect() as c:
    print("SELECT 1 ->", c.execute(text("SELECT 1")).scalar())


ImportError: cannot import name 'load_sql_settings' from 'graphfw.core.config' (unknown location)

## 2) Windows Integrated / Trusted Connection — `connections.sql_trusted`

In [ ]:
connect_and_check("connections.sql_trusted")  # läuft unter Identität des Prozesses

## 3) Azure AD Password — `connections.sql_aad_password`

In [ ]:
connect_and_check("connections.sql_aad_password")

## 4) Azure AD Integrated — `connections.sql_aad_integrated`

In [ ]:
connect_and_check("connections.sql_aad_integrated")  # SSO; funktioniert nur, wenn Client/Session dafür geeignet ist

## 5) Azure AD Interactive — `connections.sql_aad_interactive`

In [ ]:
RUN_INTERACTIVE = False  # auf True setzen, wenn ein UI-Login möglich ist
if RUN_INTERACTIVE:
    connect_and_check("connections.sql_aad_interactive")
else:
    print("Übersprungen (RUN_INTERACTIVE=False)")

## 6) Azure AD Managed Identity — `connections.sql_aad_msi`

In [ ]:
RUN_MSI = False  # auf True setzen, wenn auf Azure-Host mit MI ausgeführt
if RUN_MSI:
    connect_and_check("connections.sql_aad_msi")
else:
    print("Übersprungen (RUN_MSI=False)")

## 7) Azure AD Service Principal (Client Secret) — `connections.sql_aad_sp`

In [ ]:
connect_and_check("connections.sql_aad_sp")

### Hinweis zu Treiber & Verschlüsselung
- Bei **ODBC Driver 18** ist `Encrypt=yes` Standard. In Testumgebungen hilft oft `TrustServerCertificate=yes` zusätzlich (per `params`).
- Die `auth`-Shorthands in der `config.json` setzen die nötigen ODBC-Parameter automatisch (siehe `graphfw.core.config.sql_config`).